In [33]:
import requests
from time import sleep
import pandas as pd
from os import mkdir, path

In [2]:
addresses = {
    "UniswapV3Factory": "0x1F98431c8aD98523631AE4a59f267346ea31F984",
    "Multicall2": "0x5BA1e12693Dc8F9c48aAD8770482f4739bEeD696",
    "ProxyAdmin": "0xB753548F6E010e7e680BA186F9Ca1BdAB2E90cf2",
    "TickLens": "0xbfd8137f7d1516D3ea5cA83523914859ec47F573",
    "Quoter": "0xb27308f9F90D607463bb33eA1BeBb41C27CE5AB6",
    "SwapRouter": "0xE592427A0AEce92De3Edee1F18E0157C05861564",
    "NFTDescriptor": "0x42B24A95702b9986e82d421cC3568932790A48Ec",
    "NonfungibleTokenPositionDescriptor": "0x91ae842A5Ffd8d12023116943e72A606179294f3",
    "TransparentUpgradeableProxy": "0xEe6A57eC80ea46401049E92587E52f5Ec1c24785",
    "NonfungiblePositionManager": "0xC36442b4a4522E871399CD717aBDD847Ab11FE88",
    "V3Migrator": "0xA5644E29708357803b5A882D272c41cC0dF92B34"
}

api_key = ""

In [5]:
def get_url(address):
    return f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={address}&apikey={api_key}"

In [44]:
def get_data_from_address(address):
    url = get_url(address)
    res = requests.get(url)
    source = res.json()['result'][0]
    true = True
    false = False
    abi = source['ABI']
    if (source['SourceCode'][:2] == ('{{')) and (source['SourceCode'][-2:] == ('}}')):
        source = eval(source['SourceCode'][1:-1])
        if 'sources' in source:
            source = source['sources']
        elif 'source' in source:
            source = source['source']
    elif (source['SourceCode'][0] == '{') and (source['SourceCode'][-1] == '}'):
        source = eval(source['SourceCode'])
        if 'sources' in source:
            source = source['sources']
        elif 'source' in source:
            source = source['source']
    else:
        source = {source['ContractName']: {"content":source['SourceCode']}}
    return source, abi


In [45]:
res_dict = {}

In [46]:
while True:
    try:
        for k, v in addresses.items():
            print(k)
            if k in res_dict:
                curr_source, curr_abi = res_dict[k]['source'], res_dict[k]['abi']
            else:
                curr_source, curr_abi = get_data_from_address(v)
                res_dict[k] = {'source': curr_source, 'abi': curr_abi}
                sleep(5)
        break
    except:
        print("Sleeping")
        sleep(60)

UniswapV3Factory
Multicall2
ProxyAdmin
TickLens
Quoter
SwapRouter
Sleeping
UniswapV3Factory
Multicall2
ProxyAdmin
TickLens
Quoter
SwapRouter
NFTDescriptor
NonfungibleTokenPositionDescriptor
TransparentUpgradeableProxy
NonfungiblePositionManager
V3Migrator


In [ ]:
res_dict.keys

In [97]:
abi_df = pd.DataFrame(columns=["ContractName","ABI"])
for k,v in res_dict.items():
    if not path.exists(k):
        mkdir(k)
    abi_df = abi_df.append({"ContractName": k,"ABI": v['abi']},ignore_index=True)
    print(k)
    vs = v['source']
    for ki, vi in vs.items():
        kis = ki.split('/')
        if len(kis) > 1:
            for i in range(len(kis)-1):
                to_create = k + '/' + '/'.join(kis[:i+1])
                if not path.exists(to_create):
                    mkdir(to_create)
        fileName = k + '/' + ki
        if not (fileName[-4:] == ".sol"):
            fileName += ".sol"
        print(fileName)
        with open(fileName, "w") as text_file:
            out = vi['content'].replace("unicode'⚠️","'")
            print(out, file=text_file)

UniswapV3Factory
UniswapV3Factory/contracts/UniswapV3Factory.sol
UniswapV3Factory/contracts/interfaces/IUniswapV3Factory.sol
UniswapV3Factory/contracts/UniswapV3PoolDeployer.sol
UniswapV3Factory/contracts/NoDelegateCall.sol
UniswapV3Factory/contracts/UniswapV3Pool.sol
UniswapV3Factory/contracts/interfaces/IUniswapV3PoolDeployer.sol
UniswapV3Factory/contracts/interfaces/IUniswapV3Pool.sol
UniswapV3Factory/contracts/libraries/LowGasSafeMath.sol
UniswapV3Factory/contracts/libraries/SafeCast.sol
UniswapV3Factory/contracts/libraries/Tick.sol
UniswapV3Factory/contracts/libraries/TickBitmap.sol
UniswapV3Factory/contracts/libraries/Position.sol
UniswapV3Factory/contracts/libraries/Oracle.sol
UniswapV3Factory/contracts/libraries/FullMath.sol
UniswapV3Factory/contracts/libraries/FixedPoint128.sol
UniswapV3Factory/contracts/libraries/TransferHelper.sol
UniswapV3Factory/contracts/libraries/TickMath.sol
UniswapV3Factory/contracts/libraries/LiquidityMath.sol
UniswapV3Factory/contracts/libraries/Sqrt

In [99]:
abi_df.to_csv('abi_df', index=False)

In [62]:
res_dict["NFTDescriptor"]['source']['contracts/libraries/NFTDescriptor.sol']['content'][5287:5288]

'⚠'

In [68]:
res_dict["NFTDescriptor"]['source']['contracts/libraries/NFTDescriptor.sol']['content'][5267:5298].replace('⚠','')

"            unicode'️ DISCLAIM"

In [79]:
'U+26A0' in out[5267:5298]

False

In [78]:
'U+26A0'

'U+26A0'

In [85]:
with open('dei.sol', "w") as text_file:
    print(out.encode().decode(), file=text_file)

UnicodeEncodeError: 'charmap' codec can't encode character '\ufe0f' in position 5436: character maps to <undefined>

In [94]:
out[5279:5289]

"unicode'⚠️"

In [96]:
out.replace("unicode'⚠️","")

'// SPDX-License-Identifier: UNLICENSED\npragma solidity >=0.7.0;\npragma abicoder v2;\n\nimport \'@uniswap/v3-core/contracts/interfaces/IUniswapV3Pool.sol\';\nimport \'@uniswap/v3-core/contracts/libraries/TickMath.sol\';\nimport \'@uniswap/v3-core/contracts/libraries/BitMath.sol\';\nimport \'@uniswap/v3-core/contracts/libraries/FullMath.sol\';\nimport \'@openzeppelin/contracts/utils/Strings.sol\';\nimport \'@openzeppelin/contracts/math/SafeMath.sol\';\nimport \'@openzeppelin/contracts/math/SignedSafeMath.sol\';\nimport \'base64-sol/base64.sol\';\nimport \'./HexStrings.sol\';\nimport \'./NFTSVG.sol\';\n\nlibrary NFTDescriptor {\n    using TickMath for int24;\n    using Strings for uint256;\n    using SafeMath for uint256;\n    using SafeMath for uint160;\n    using SafeMath for uint8;\n    using SignedSafeMath for int256;\n    using HexStrings for uint256;\n\n    uint256 constant sqrt10X128 = 1076067327063303206878105757264492625226;\n\n    struct ConstructTokenURIParams {\n        uin

In [74]:
out

'// SPDX-License-Identifier: UNLICENSED\npragma solidity >=0.7.0;\npragma abicoder v2;\n\nimport \'@uniswap/v3-core/contracts/interfaces/IUniswapV3Pool.sol\';\nimport \'@uniswap/v3-core/contracts/libraries/TickMath.sol\';\nimport \'@uniswap/v3-core/contracts/libraries/BitMath.sol\';\nimport \'@uniswap/v3-core/contracts/libraries/FullMath.sol\';\nimport \'@openzeppelin/contracts/utils/Strings.sol\';\nimport \'@openzeppelin/contracts/math/SafeMath.sol\';\nimport \'@openzeppelin/contracts/math/SignedSafeMath.sol\';\nimport \'base64-sol/base64.sol\';\nimport \'./HexStrings.sol\';\nimport \'./NFTSVG.sol\';\n\nlibrary NFTDescriptor {\n    using TickMath for int24;\n    using Strings for uint256;\n    using SafeMath for uint256;\n    using SafeMath for uint160;\n    using SafeMath for uint8;\n    using SignedSafeMath for int256;\n    using HexStrings for uint256;\n\n    uint256 constant sqrt10X128 = 1076067327063303206878105757264492625226;\n\n    struct ConstructTokenURIParams {\n        uin